In [1]:
import os
import fnmatch
import json
import dateutil.parser
import glob
import pandas as pd

In [28]:
methods = []
for root, dirnames, filenames in os.walk('/pod/pstore/groups/treehouse/treeshop/'):
    for filename in fnmatch.filter(filenames, 'methods.json'):
        methods.append(json.loads(open(os.path.join(root, filename)).read()))

In [44]:
methods = [json.loads(open(m).read()) for m in
           glob.glob("/pod/pstore/groups/treehouse/archive/downstream/*/secondary/*/methods.json") if "TEST" not in m]

In [45]:
pd.DataFrame([{"id": m["sample_id"], 
               "pipeline": m["pipeline"]["docker"]["url"],
               "duration": (dateutil.parser.parse(m["end"]) 
                            - dateutil.parser.parse(m["start"])).total_seconds()/(60*60)}
 for m in methods]).groupby("pipeline").min()

,duration,id
pipeline,,
https://hub.docker.com/alpine,0.006154,THR30_0805_S01
https://hub.docker.com/r/ucsctreehouse/fusion,0.635304,THR30_0804_S01
https://hub.docker.com/r/ucsctreehouse/mini-var-call,0.052230,THR30_0804_S01
https://quay.io/ucsc_cgl/rnaseq-cgl-pipeline,9.060947,THR30_0804_S01


In [52]:
stats = pd.DataFrame(columns=["Expression w/QC", "Expression w/o QC", 
                              "Delta", "Fusion", "Variants", "All w/QC", "Cost"])
for m in methods:
    withqc = json.loads(open(path).read())

    duration_withqc = (dateutil.parser.parse(withqc["end"]) 
                       - dateutil.parser.parse(withqc["start"])).total_seconds()/(60*60)

    duration_noqc = (dateutil.parser.parse(noqc["end"]) 
                       - dateutil.parser.parse(noqc["start"])).total_seconds()/(60*60)

    path = "/pod/pstore/groups/treehouse/archive/downstream/{}/fusions/methods.json".format(noqc["sample_id"])
    fusions = json.loads(open(path).read())
    fusions_duration = (dateutil.parser.parse(fusions["end"]) 
                       - dateutil.parser.parse(fusions["start"])).total_seconds()/(60*60)

    path = "/pod/pstore/groups/treehouse/archive/downstream/{}/variants/methods.json".format(noqc["sample_id"])
    variants = json.loads(open(path).read())
    variants_duration = (dateutil.parser.parse(variants["end"]) 
                       - dateutil.parser.parse(variants["start"])).total_seconds()/(60*60)

    stats.loc[noqc["sample_id"]] = [duration_withqc, duration_noqc, 
                                    duration_withqc - duration_noqc,
                                    fusions_duration, variants_duration, 
                                    duration_withqc + fusions_duration + variants_duration,
                                    (duration_withqc + fusions_duration + variants_duration) * 0.8]

stats        

,Expression w/QC,Expression w/o QC,Delta,Fusion,Variants,All w/QC,Cost
TH27_0695_S01,18.349867,9.838063,8.511804,1.175474,0.125489,19.650831,15.720665
TH27_0696_S01,16.489687,8.060334,8.429353,1.708181,0.107113,18.304981,14.643984
TH27_0698_S01,18.682528,10.287316,8.395212,1.141220,0.123743,19.947491,15.957993
TH27_0697_S01,13.690776,7.338090,6.352686,0.988169,0.123118,14.802063,11.841650
TH27_0700_S01,14.919778,7.743452,7.176326,0.997348,0.072306,15.989432,12.791546
TH27_0699_S02,17.022716,8.676143,8.346573,2.049321,0.075315,19.147351,15.317881
TH27_0699_S01,15.453424,7.496854,7.956570,0.990865,0.066393,16.510682,13.208546
TH27_0701_S01,15.819080,8.136608,7.682472,1.385793,0.073327,17.278200,13.822560
TH27_0704_S01,18.628554,9.141510,9.487044,1.647023,0.080287,20.355863,16.284691
TH27_0703_S01,15.732345,8.604336,7.128009,1.123697,0.068373,16.924416,13.539532


In [55]:
summary = stats.describe()
summary.index.name = "stat"
summary

,Expression w/QC,Expression w/o QC,Delta,Fusion,Variants,All w/QC,Cost
stat,,,,,,,
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,16.266004,8.404679,7.861325,1.364550,0.089116,17.719670,14.175736
std,1.606614,0.980005,0.848109,0.377563,0.023935,1.739799,1.391839
min,13.690776,7.093771,6.352686,0.988169,0.065624,14.802063,11.841650
25%,15.321501,7.681803,7.164247,1.092110,0.071323,16.727396,13.381917
50%,15.775713,8.288139,7.894265,1.204622,0.077801,17.101648,13.681319
75%,17.354504,8.792485,8.403747,1.662312,0.111114,19.273221,15.418577
max,18.682528,10.287316,9.487044,2.049321,0.125489,20.355863,16.284691
